In [ ]:
from functions import *
import pandas as pd
import numpy as np
import datetime as dt
import math
import networkx as nx
#networkx 2.0
from networkx.drawing.nx_agraph import graphviz_layout
import pygraphviz
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
from MST import importdata, MST, draw_network
from sklearn.metrics import adjusted_rand_score
import igraph as ig

In [ ]:
price, log_ret = importdata("sp96price.csv")
dic = rolling_corr(log_ret, 250, "2017-01-31", "2015-12-30", 10)
ret = price/price.shift(1)
ret = ret.iloc[1:]

In [ ]:
#test for the hypergeometric test
nodenames = np.array(importdata("SP100_prices.csv")[0].columns)
clusters = clustersGirvanNewman2
df = importdata("SP100_prices.csv")[1]
sorteddates = sorted(clusters.keys(), key=lambda d: map(int, d.split('-')))
HGT = np.empty(len(sorteddates)-1)
enddate = "2017-01-24"
startdate = '2015-12-30'   
end = int(np.where(df.index==enddate)[0])
start = int(np.where(df.index==startdate)[0])
total_trees = MST(filename="SP100_prices.csv", window=end-start+1, enddate="2017-01-24", startdate='2015-12-30',space=1)
total_clusters = tuple(nx.girvan_newman(total_trees[sorteddates[0]]))[0]
# The total number of elements within all the clusters is N for the whole time frame
N = 0
for cluster in total_clusters:
    N = N + len(cluster)
    
new_current_date_clusters = {}
for i in range(0,len(sorteddates)):
    current_date_clusters = clusters[sorteddates[i]]
    current_cluster_list = {}
    for j in range(0, len(total_clusters)):
        current_cluster = tuple()
        for k in range(0, len(current_date_clusters)):
            overlap = len((current_date_clusters[k]).intersection(total_clusters[j]))
            p = hypergeom.pmf(overlap, N, len(current_date_clusters[k]), len(total_clusters[j])) 
            if((p < 0.01) & (len(current_cluster) < len(current_date_clusters[k]))):
                current_cluster = current_date_clusters[k]
        current_cluster_list[j] = current_cluster
    new_current_date_clusters[sorteddates[i]] = current_cluster_list

print new_current_date_clusters

In [ ]:
nodenames = np.array(importdata("SP100_prices.csv")[0].columns)
trees = MST()
sorteddates = sorted(trees.keys(), key=lambda d: map(int, d.split('-')))

In [ ]:
IGtrees = {}
for key in sorteddates:
    IGtrees[key] = NXtoIG(trees[key])

Newman - community_leading_eigenvector()  
Newman Girvan - nx.girvan_newman()  
Newman Clauset - community_fastgreedy()

In [1]:
clustersGirvanNewman2 = {}
clustersGirvanNewman4 = {}
clustersGirvanNewman8 = {}
clustersGirvanNewman16 = {}
for key in sorteddates:
    C = tuple(nx.girvan_newman(trees[key]))
    clustersGirvanNewman2[key] = C[0]
    clustersGirvanNewman4[key] = C[2]
    clustersGirvanNewman8[key] = C[6]
    clustersGirvanNewman16[key] = C[14]

NameError: name 'sorteddates' is not defined

In [ ]:
ig.arpack_options.maxiter=500000
clustersNewman = {}
clustersClausetNewman = {}
for key in sorteddates:
    C = IGtrees[key].community_leading_eigenvector(weights="weight")
    clustersNewman[key] = list(C)
    for i in range(0,len(C)):
        clustersNewman[key][i] = [IGtrees[key].vs["name"][j] for j in C[i]]
    C = IGtrees[key].community_fastgreedy(weights="weight").as_clustering()
    clustersClausetNewman[key] = list(C)
    for i in range(0,len(C)):
        clustersClausetNewman[key][i] = [IGtrees[key].vs["name"][j] for j in C[i]]